In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
# %pip install geopandas

In [ ]:
# import geopandas as gpd
# from geopandas import GeoDataFrame
# from shapely.geometry import Point

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data preprocessing

## Data merging
Preprocess data and create a list of reasonable OD pairs.

In [ ]:
!ls "/content/drive/My Drive/CMAP data"

chicago.dbf			directions_walking.json
chicago.prj			directions_walking_old2.json
chicago.shp			gps_place.csv
chicago.shx			household.csv
directions_bicycling.json	location.csv
directions_bicycling_old2.json	person.csv
directions_driving.json		place.csv
directions_driving_old2.json	X_normalized_old.csv
directions_transit.json		X_unnormalized_old.csv
directions_transit_old2.json


In [ ]:
place = pd.read_csv('/content/drive/My Drive/CMAP data/gps_place.csv', low_memory=False)
loc = pd.read_csv('/content/drive/My Drive/CMAP data/location.csv', low_memory=False)

In [ ]:
pl = place.query('mode_imputed == [1, 2, 3, 4] and travtime > 0 and distance > 0')
pl = pl[['sampno', 'perno', 'locno', 'arrtime', 'deptime', 'travtime', 'distance', 'mode_imputed', 'fare', 'pkamt']]
l = loc.query('latitude != -9')[['sampno', 'locno', 'latitude', 'longitude']]

pl_l = pd.merge(pl, l, left_on=['sampno', 'locno'], right_on=['sampno', 'locno'], how='inner')

In [ ]:
pers = pd.read_csv('/content/drive/My Drive/CMAP data/person.csv', low_memory=False)
hh = pd.read_csv('/content/drive/My Drive/CMAP data/household.csv', low_memory=False)

In [ ]:
p = pers.query('age > 0')[['sampno', 'perno', 'age']]
h = hh.query('hhinc > 0')[['sampno', 'hhveh', 'hhinc']]

pers_hh = pd.merge(p, h, left_on='sampno', right_on='sampno', how='inner')

In [ ]:
data = pd.merge(pers_hh, pl_l, left_on=['sampno', 'perno'], right_on=['sampno', 'perno'], how='inner')
data = data.rename(columns={'latitude': 'arr_lat', 'longitude': 'arr_lon'})

In [ ]:
for i in range(1, len(data)):
    if data.loc[i-1, 'sampno'] == data.loc[i, 'sampno'] and data.loc[i-1, 'perno'] == data.loc[i, 'perno']:
        data.loc[i, 'dep_lat'] = data.loc[i-1, 'arr_lat']
        data.loc[i, 'dep_lon'] = data.loc[i-1, 'arr_lon']
    else:
        data.loc[i, 'dep_lat'] = np.nan
        data.loc[i, 'dep_lon'] = np.nan

In [ ]:
df = data.query('dep_lat != arr_lat and dep_lon != arr_lon').dropna().reset_index(drop=True)

In [ ]:
od = []

for i in range(len(df)):
    pair = (str(df.loc[i, 'dep_lat'])+','+str(df.loc[i, 'dep_lon']), str(df.loc[i, 'arr_lat'])+','+str(df.loc[i, 'arr_lon']))
    if pair not in od:
        od.append(pair)

len(od)

26363

## Plotting data coverage
Plot data coverage of the first 5000 OD pairs.

In [ ]:
# dep_lat = [float(od[i][0].split(',')[0]) for i in range(len(od))]
# dep_lon = [float(od[i][0].split(',')[1]) for i in range(len(od))]
# arr_lat = [float(od[i][1].split(',')[0]) for i in range(len(od))]
# arr_lon = [float(od[i][1].split(',')[1]) for i in range(len(od))]

# od_smp = od[:5000]
# dep_lat_smp = [float(od_smp[i][0].split(',')[0]) for i in range(5000)]
# dep_lon_smp = [float(od_smp[i][0].split(',')[1]) for i in range(5000)]
# arr_lat_smp = [float(od_smp[i][1].split(',')[0]) for i in range(5000)]
# arr_lon_smp = [float(od_smp[i][1].split(',')[1]) for i in range(5000)]

In [ ]:
# def od_plot_all(lon, lat, tag, color):
#     geo = [Point(xy) for xy in zip(lon, lat)]
#     gdf = GeoDataFrame(geometry=geo)
#     gdf.crs = 'epsg:4326'
    
#     data = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#     ax = data.plot(figsize=(8, 7), color='gray', edgecolor='black', alpha=.3)
#     ax.set_xlim(-126, -66)
#     ax.set_ylim(24, 50)
#     gdf.plot(ax=ax, marker='.', color=color, alpha=.3, markersize=10, label=tag)
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel('Latitude')
#     ax.legend()

In [ ]:
# od_plot_all(dep_lon_smp, dep_lat_smp, 'Origin (Sample)', 'red')

In [ ]:
# def od_plot(lon, lat, tag, color):
#     geo = [Point(xy) for xy in zip(lon, lat)]
#     gdf = GeoDataFrame(geometry=geo)
#     gdf.crs = 'epsg:4326'
    
#     data = gpd.read_file('/content/drive/My Drive/CMAP data/chicago.shp')
#     ax = data.plot(figsize=(8, 7), color='gray', edgecolor='black', alpha=.3)
#     ax.set_xlim(-88, -87.4)
#     ax.set_ylim(41.6, 42.1)
#     gdf.plot(ax=ax, marker='.', color=color, alpha=.3, markersize=10, label=tag)
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel('Latitude')
#     ax.legend()

In [ ]:
# od_plot(dep_lon_smp, dep_lat_smp, 'Origin (Sample)', 'red')
# od_plot(arr_lon_smp, arr_lat_smp, 'Destination (Sample)', 'green')

In [ ]:
# od_plot(dep_lon, dep_lat, 'Origin', 'red')
# od_plot(arr_lon, arr_lat, 'Destination', 'green')

# Data completion

## Google Directions API
Use Google API to extract distances and durations for the first 5000 OD pairs.

In [ ]:
# usr_key_google = ''
# txt = []

# for i in range(5000):
#     if i % 100 == 0:
#         print(f'Ite = {i}')
#     o_coor = od[i][0]
#     d_coor = od[i][1]
#     url = 'https://maps.googleapis.com/maps/api/directions/json?' + \
#           'origin=' + o_coor + '&destination=' + d_coor + \
#           '&key=' + usr_key_google + \
#           '&mode=walking'
#     res = requests.get(url)
#     text = json.loads(res.content)
#     txt.append(text)

# with open('directions_walking.json', 'a') as f:
#     json.dump(txt, f, indent=2)

In [ ]:
with open('/content/drive/My Drive/CMAP data/directions_driving.json', 'r') as f:
    data1 = json.load(f)
txt1 = data1

with open('/content/drive/My Drive/CMAP data/directions_walking.json', 'r') as f:
    data2 = json.load(f)
txt2 = data2

with open('/content/drive/My Drive/CMAP data/directions_transit.json', 'r') as f:
    data3 = json.load(f)
txt3 = data3

with open('/content/drive/My Drive/CMAP data/directions_bicycling.json', 'r') as f:
    data4 = json.load(f)
txt4 = data4

In [ ]:
dt1, tt1 = [], []
dt2, tt2 = [], []
dt3, tt3 = [], []
dt4, tt4 = [], []

for i in range(5000):
    try:
        dt1.append(txt1[i]['routes'][0]['legs'][0]['distance']['value'] / 1609)
        tt1.append(txt1[i]['routes'][0]['legs'][0]['duration']['value'] / 60)
    except:
        dt1.append(np.nan)
        tt1.append(np.nan)
    try:
        dt2.append(txt2[i]['routes'][0]['legs'][0]['distance']['value'] / 1609)
        tt2.append(txt2[i]['routes'][0]['legs'][0]['duration']['value'] / 60)
    except:
        dt2.append(np.nan)
        tt2.append(np.nan)
    try:
        dt3.append(txt3[i]['routes'][0]['legs'][0]['distance']['value'] / 1609)
        tt3.append(txt3[i]['routes'][0]['legs'][0]['duration']['value'] / 60)
    except:
        dt3.append(np.nan)
        tt3.append(np.nan)
    try:
        dt4.append(txt4[i]['routes'][0]['legs'][0]['distance']['value'] / 1609)
        tt4.append(txt4[i]['routes'][0]['legs'][0]['duration']['value'] / 60)
    except:
        dt4.append(np.nan)
        tt4.append(np.nan)

In [ ]:
for i in df.index:
    pair = (str(df.loc[i, 'dep_lat'])+','+str(df.loc[i, 'dep_lon']), str(df.loc[i, 'arr_lat'])+','+str(df.loc[i, 'arr_lon']))
    idx = od.index(pair)
    try:
        df.loc[i, 'dist_auto'] = dt1[idx]
        df.loc[i, 'time_auto'] = tt1[idx]
    except:
        df.loc[i, 'dist_auto'] = np.nan
        df.loc[i, 'time_auto'] = np.nan
    try:
        df.loc[i, 'dist_walk'] = dt2[idx]
        df.loc[i, 'time_walk'] = tt2[idx]
    except:
        df.loc[i, 'dist_walk'] = np.nan
        df.loc[i, 'time_walk'] = np.nan
    try:
        df.loc[i, 'dist_train'] = dt3[idx]
        df.loc[i, 'time_train'] = tt3[idx]
    except:
        df.loc[i, 'dist_train'] = np.nan
        df.loc[i, 'time_train'] = np.nan
    try:
        df.loc[i, 'dist_bike'] = dt4[idx]
        df.loc[i, 'time_bike'] = tt4[idx]
    except:
        df.loc[i, 'dist_bike'] = np.nan
        df.loc[i, 'time_bike'] = np.nan

We find that in many cases, OD distances in the CMAP data are very different from what Google API provides. Thus, we calculate the great-circle distance for all OD pairs and study only "reasonable" trips.

In [ ]:
from geopy import distance

df['dist'] = [distance.distance((df.loc[i, 'dep_lat'], df.loc[i, 'dep_lon']),
                                (df.loc[i, 'arr_lat'], df.loc[i, 'arr_lon'])).miles for i in range(len(df))]
df1 = df.query('dist <= 30').copy()
df1.shape

(33404, 26)

In [ ]:
place1 = pd.read_csv('/content/drive/My Drive/CMAP data/place.csv', low_memory=False)

df2 = pd.merge(df1, place1[['sampno', 'perno', 'locno', 'plaza_total']].drop_duplicates(subset=['sampno', 'perno', 'locno']),
               left_on=['sampno', 'perno', 'locno'], right_on=['sampno', 'perno', 'locno'], how='left')

In [ ]:
# Deal with transit and driving costs
df2['cost_train'] = df2['fare']
df2.loc[df2['mode_imputed'].isin([1, 2, 4]), 'cost_train'] = 0
df2.loc[df2['cost_train'] <= 0, 'cost_train'] = np.nan

df2['cost_parking'] = df2['pkamt']
df2.loc[df2['mode_imputed'].isin([2, 3, 4]), 'cost_parking'] = 0
df2.loc[(df2['cost_parking'] <= 0) | (df2['cost_parking'] > 50), 'cost_parking'] = np.nan

df2['cost_toll'] = df2['plaza_total']
df2.loc[df2['mode_imputed'].isin([2, 3, 4]), 'cost_toll'] = 0
df2.loc[df2['cost_toll'] < 0, 'cost_toll'] = np.nan

## Data imputation
Perform the $K$-nearest neighbors algorithm on the missing data, and finally export the processed dataset.

In [ ]:
x = df2[['travtime', 'distance', 'dist_auto', 'time_auto', 'dist_walk', 'time_walk', 'dist_train', 'time_train',
         'dist_bike', 'time_bike', 'cost_train', 'cost_parking', 'cost_toll']]
imputer = KNNImputer(n_neighbors=6, weights='uniform')
x_imputed = imputer.fit_transform(x)

In [ ]:
df2[['travtime', 'distance', 'dist_auto', 'time_auto', 'dist_walk', 'time_walk', 'dist_train', 'time_train',
     'dist_bike', 'time_bike', 'cost_train', 'cost_parking', 'cost_toll']] = x_imputed
df2['cost_auto'] = df2['cost_toll'] + df2['cost_parking']

In [ ]:
df2[['dist_auto', 'time_auto', 'dist_walk', 'time_walk', 'dist_train', 'time_train',
     'dist_bike', 'time_bike', 'cost_train', 'cost_auto']].describe()

,dist_auto,time_auto,dist_walk,time_walk,dist_train,time_train,dist_bike,time_bike,cost_train,cost_auto
count,33404.000000,33404.000000,33404.000000,33404.000000,33404.000000,33404.000000,33404.000000,33404.000000,33404.000000,33404.000000
mean,6.450315,13.817148,5.662759,113.883035,8.018903,56.845839,6.276666,34.856698,2.386652,8.746128
std,6.408127,8.125875,5.676216,113.681337,8.893555,51.349741,6.410197,34.064499,0.751449,5.806160
min,0.139838,0.550000,0.139838,3.183333,0.139838,3.183333,0.139838,1.116667,0.250000,0.250000
25%,2.243837,8.100000,1.922105,38.827778,2.126683,26.688889,2.135799,12.798611,1.833333,3.800000
50%,4.249016,11.555556,3.790087,76.458333,4.764346,40.883333,4.088668,23.172222,2.208333,6.850000
75%,8.019500,17.272222,7.110421,142.958333,10.255542,66.031250,7.805055,42.747222,2.541667,13.595833
max,45.170292,56.666667,40.921069,801.083333,135.200124,741.083333,45.051585,246.150000,10.000000,50.000000


## Data export

In [ ]:
X = df2[['age', 'hhveh', 'hhinc', 'dist_auto', 'time_auto', 'dist_walk', 'time_walk', 'dist_train', 'time_train',
         'dist_bike', 'time_bike', 'cost_train', 'cost_auto']]
X.to_csv('/content/drive/My Drive/CMAP data/X_unnormalized.csv')
y = df2['mode_imputed'].add(-1)
y.to_csv('/content/drive/My Drive/CMAP data/y.csv')

X1 = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns)
X1.to_csv('/content/drive/My Drive/CMAP data/X_normalized.csv')
print(X.shape)

(33404, 13)
